# **Co2 Emissions and Economic**

```
📌 https://data.worldbank.org/indicator/NY.GDP.PCAP.CD 
📌 https://data.worldbank.org/indicator/EN.ATM.CO2E.PC 
📌 https://github.com/owid/energy-data 
📌 https://carbonpricingdashboard.worldbank.org/map_data 
```

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno
!pip install squarify
import squarify as sq

import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as pyoff
import warnings # Ignores any warning
warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **Load Data**

In [25]:
# GDPPerCapita.csv
gdp = pd.read_csv("https://drive.google.com/uc?id=1rBgi4F_R9EhYerayeFvaRaRCPQV6N_IY")
gdp.head()

,Country Name,Year,GDP
0,Aruba,1960,0.0
1,Aruba,1961,0.0
2,Aruba,1962,0.0
3,Aruba,1963,0.0
4,Aruba,1964,0.0


In [26]:
# CO2EmissionsPerCapita.csv
co2pc = pd.read_csv("https://drive.google.com/uc?id=136cdpMhIX_WKyg5A_FKAjdDspog_Rc7p")
co2pc.head()

,Country Name,Attribute,Value
0,Aruba,1960,0.0
1,Aruba,1961,0.0
2,Aruba,1962,0.0
3,Aruba,1963,0.0
4,Aruba,1964,0.0


In [27]:
# ElectricityGeneratedYear.csv
elecdt = pd.read_csv("https://drive.google.com/uc?id=1f3LClVnVBSjExxdvsl5wSQBv3jwxF1G-")
elecdt.head()

,Entity,Year,Generated_Electricity,Fossil_Energy,Nuclear_Energy,Renewable_Electricity
0,Afghanistan,2000,0.467,0.155,0.0,0.312
1,Afghanistan,2001,0.592,0.094,0.0,0.498
2,Afghanistan,2002,0.687,0.132,0.0,0.555
3,Afghanistan,2003,0.939,0.309,0.0,0.630
4,Afghanistan,2004,0.894,0.329,0.0,0.565


In [28]:
# AnnualCOEmissionsbyRegion.csv
co2ann = pd.read_csv("https://drive.google.com/uc?id=1LDi87mDkdnCkl6DN_CqZw9NZprYQGUch")
co2ann.head()

,Entity,Code,Year,Annual CO2 emissions (zero filled)
0,Afghanistan,AFG,1750,0
1,Afghanistan,AFG,1751,0
2,Afghanistan,AFG,1752,0
3,Afghanistan,AFG,1753,0
4,Afghanistan,AFG,1754,0


# **Exploratory Data Analysis and Data Cleaning**

In [29]:
gdp.rename(columns={"Country Name":"Country"}, inplace=True)

In [30]:
co2pc.rename(columns={
    "Value":"Co2_p",
    "Attribute":"Year",
    "Country Name":"Country"
}, inplace=True)

In [31]:
gco_join = gdp.merge(co2pc, how='inner', on=['Country', 'Year'])
gco_join.head()

,Country,Year,GDP,Co2_p
0,Aruba,1960,0.0,0.0
1,Aruba,1961,0.0,0.0
2,Aruba,1962,0.0,0.0
3,Aruba,1963,0.0,0.0
4,Aruba,1964,0.0,0.0


In [32]:
energygb = elecdt.groupby("Year").agg({
    "Fossil_Energy":"sum",
    "Nuclear_Energy":"sum",
    "Renewable_Electricity":"sum"
}).reset_index()
energygb.head()

,Year,Fossil_Energy,Nuclear_Energy,Renewable_Electricity
0,1985,6152.132496,1089.073749,1517.552392
1,1986,6302.889876,1142.099574,1548.361550
2,1987,6614.661098,1262.384033,1525.242674
3,1988,6846.083940,1395.379394,1563.725057
4,1989,7299.616570,1422.821226,1590.776863


In [33]:
energy = pd.melt(energygb, id_vars=['Year'], value_vars=['Fossil_Energy', 'Nuclear_Energy','Renewable_Electricity'],
        var_name='Energy Type', value_name='Energy').sort_values(["Year","Energy Type"]).reset_index(drop=True)
energy.head()

,Year,Energy Type,Energy
0,1985,Fossil_Energy,6152.132496
1,1985,Nuclear_Energy,1089.073749
2,1985,Renewable_Electricity,1517.552392
3,1986,Fossil_Energy,6302.889876
4,1986,Nuclear_Energy,1142.099574


In [34]:
co2ann.rename(columns={'Annual CO2 emissions (zero filled)':"Co2Emissions"}, inplace=True)

In [35]:
co2ann = co2ann.groupby("Year").agg({
    "Co2Emissions":"sum"
}).reset_index()

co2ann = co2ann[co2ann["Year"]>=1900]
co2ann

,Year,Co2Emissions
150,1900,8322746133
151,1901,8503958746
152,1902,8661383395
153,1903,9329177584
154,1904,9463532757
...,...,...
266,2016,124705642926
267,2017,126304712027
268,2018,128452535469
269,2019,128422888228


# **Correlation**

## **Carbon Tax Implemented**

In [36]:
# Correlation on GDP and Co2 from a Country with 
# Carbon Tax Implemented (United Kingdom)

uk_cor = gco_join[gco_join["Country"] == "United Kingdom"][["GDP", "Co2_p", "Year"]].reset_index(drop=True)

from scipy.stats import pearsonr

# calculate Pearson's correlation
corr, _ = pearsonr(uk_cor["GDP"], uk_cor["Co2_p"])
print('Pearsons correlation: %.3f' % corr)

Pearsons correlation: -0.889


correlation coefficient = -0.889 which proves strong negative correlation of GDP and Co2 Values


In [37]:
# Correlation on GDP and Co2 from a Country with 
# Carbon Tax Implemented (Sweden)

swe_cor = gco_join[gco_join["Country"] == "Sweden"][["GDP", "Co2_p", "Year"]].reset_index(drop=True)

from scipy.stats import pearsonr

# calculate Pearson's correlation
corr, _ = pearsonr(swe_cor["GDP"], swe_cor["Co2_p"])
print('Pearsons correlation: %.3f' % corr)

Pearsons correlation: -0.830


correlation coefficient = -0.830 which proves strong negative correlation of GDP and Co2 Values

## **Carbon Tax not Implemented**

In [38]:
# Correlation on GDP and Co2 from a Country that 
# haven't implemented carbon tax (Indonesia)

indo_cor = gco_join[gco_join["Country"] == "Indonesia"][["GDP", "Co2_p", "Year"]].reset_index(drop=True)

from scipy.stats import pearsonr

# calculate Pearson's correlation
corr, _ = pearsonr(indo_cor["GDP"], indo_cor["Co2_p"])
print('Pearsons correlation: %.3f' % corr)

Pearsons correlation: 0.900


correlation coefficient = 0.900 which proves strong negative correlation of GDP and Co2 Values

In [39]:
# Correlation on GDP and Co2 from a Country that 
# haven't implemented carbon tax (India)

india_cor = gco_join[gco_join["Country"] == "India"][["GDP", "Co2_p", "Year"]].reset_index(drop=True)

from scipy.stats import pearsonr

# calculate Pearson's correlation
corr, _ = pearsonr(india_cor["GDP"], india_cor["Co2_p"])
print('Pearsons correlation: %.3f' % corr)

Pearsons correlation: 0.972


correlation coefficient = 0.972 which proves strong negative correlation of GDP and Co2 Values

# **Visualization**

**Pajak karbon** adalah pajak yang dikenakan atas pembakaran bahan bakar berbasis karbon seperti batubara, minyak, dan gas. Pajak karbon adalah sebuah kebijakan inti yang dibuat untuk mengurangi dan menghilangkan penggunaan bahan bakar fosil yang pembakarannya dapat merusak iklim.

In [40]:
new_gco_join = gco_join.copy()

# United Kingdom
new_gco_join["Co2_p_uk"] = new_gco_join["Co2_p"]*4000
new_gco_join["Co2_p_uk"] = new_gco_join["Co2_p_uk"].round(decimals = 2)

# Sweden
new_gco_join["Co2_p_sw"] = new_gco_join["Co2_p"]*4500
new_gco_join["Co2_p_sw"] = new_gco_join["Co2_p_sw"].round(decimals = 2)

# Indonesia
new_gco_join["Co2_p_indo"] = new_gco_join["Co2_p"]*2000
new_gco_join["Co2_p_indo"] = new_gco_join["Co2_p_indo"].round(decimals = 2)

# India
new_gco_join["Co2_p_india"] = new_gco_join["Co2_p"]*1000
new_gco_join["Co2_p_india"] = new_gco_join["Co2_p_india"].round(decimals = 2)

**How Effective Is Carbon Pricing?**

Let’s take a look at The United Kingdom and Sweden

Some 40 countries and more than 20 cities, states and provinces already use carbon pricing mechanisms, with more planning to implement them in the future.

https://www.worldbank.org/en/programs/pricing-carbon

**Annual GDP Vs Co2 Emissions Per Capita In the United Kingdom**

In [41]:
gco_uk = new_gco_join[(new_gco_join["Country"] =="United Kingdom") & (new_gco_join["Year"] > 1990)]

fig = go.Figure()
fig.add_trace(go.Scatter(x=gco_uk["Year"], y=gco_uk["Co2_p_uk"],
                    mode='lines+markers',
                    name='CO2/Capita'))
fig.add_trace(go.Scatter(x=gco_uk["Year"], y=gco_uk["GDP"],
                    mode='lines+markers',
                    name='GDP/Capita'))

# Edit the layout
fig.update_layout(title='Annual GDP Vs Co2 Emissions Per Capita In the United Kingdom',
                   xaxis_title='Year',
                   yaxis_title='GDP/Capita')
fig.add_annotation(x=2003.2, y=36500,
            text="EU ETS, 2005*",
            showarrow=True,arrowcolor="#636363",
            ax=-30,
            ay=-90,
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor="#ff7f0e",
            opacity=0.8,
            arrowhead=7)

fig.add_annotation(x=2013, y=44000,
            text="UK CPS, 2013**",
            showarrow=True,arrowcolor="#636363",
            ax=-30,
            ay=-90,
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor="#ff7f0e",
            opacity=0.8,
            arrowhead=7)

fig.update_layout(
        xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="""
            Year
            <br><br><sup>*The European Union Emissions Trading System (EU ETS) is a  form of Carbon Pricing.</sup>
            <br><sup>** UK Carbon Price Support (CPS) is an additonal form of Carbon Pricing.</sup>
            <br><sup>Correlation Coefficient = -0.889 (Strong Negative Correlation)</sup>
            """
            )
        )
    )

fig.show()

**Annual GDP Vs Co2 Emissions Per Capita in Sweden**

In [42]:
gco_sw = new_gco_join[(new_gco_join["Country"] =="Sweden") & (new_gco_join["Year"] > 1975)]

fig = go.Figure()
fig.add_trace(go.Scatter(x=gco_sw["Year"], y=gco_sw["Co2_p_sw"],
                    mode='lines+markers',
                    name='CO2/Capita'))
fig.add_trace(go.Scatter(x=gco_sw["Year"], y=gco_sw["GDP"],
                    mode='lines+markers',
                    name='GDP/Capita'))

# Edit the layout
fig.update_layout(title='Annual GDP Vs Co2 Emissions Per Capita In the Sweden',
                   xaxis_title='Year',
                   yaxis_title='GDP/Capita')

fig.add_annotation(x=1991, y=28000,
            text="Carbon Tax, 1991*",
            showarrow=True,arrowcolor="#636363",
            ax=0,
            ay=-90,
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor="#ff7f0e",
            opacity=0.8,
            arrowhead=1)

fig.update_layout(
        xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="""
            Year
            <br><br><sup>*Carbon Tax Implementation Started on 1991
            <br>Correlation Coefficient = -0.829 (Strong Negative Correlation)</sup>
            """
            )
        )
    )

fig.show()

After Implementation of Carbon Tax, UK and Sweden Experience Increased Economic Activity (GDP) and Decreased Carbon Emissions

**No Carbon Tax**

**Annual GDP Vs Co2 Emissions Per Capita in Indonesia**

In [43]:
gco_indo = new_gco_join[(new_gco_join["Country"] =="Indonesia") & (new_gco_join["Year"] > 1990)]

fig = go.Figure()
fig.add_trace(go.Scatter(x=gco_indo["Year"], y=gco_indo["Co2_p_indo"],
                    mode='lines+markers',
                    name='CO2/Capita'))
fig.add_trace(go.Scatter(x=gco_indo["Year"], y=gco_indo["GDP"],
                    mode='lines+markers',
                    name='GDP/Capita'))

# Edit the layout
fig.update_layout(title='Annual GDP Vs Co2 Emissions Per Capita In the Indonesia',
                   xaxis_title='Year',
                   yaxis_title='GDP/Capita')

fig.update_layout(
        xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="""
            Year
            <br><br><sup>Correlation Coefficient is 0.9001 which proves strong</sup>
            <br><sup>positive correlation  for GDP and Co2 Emissions in Indonesia.</sup>
            """
            )
        )
    )

fig.show()

**Annual GDP Vs Co2 Emissions Per Capita in India**

In [44]:
gco_india = new_gco_join[(new_gco_join["Country"] =="India") & (new_gco_join["Year"] > 1990)]

fig = go.Figure()
fig.add_trace(go.Scatter(x=gco_india["Year"], y=gco_india["Co2_p_india"],
                    mode='lines+markers',
                    name='CO2/Capita'))
fig.add_trace(go.Scatter(x=gco_india["Year"], y=gco_india["GDP"],
                    mode='lines+markers',
                    name='GDP/Capita'))

# Edit the layout
fig.update_layout(title='Annual GDP Vs Co2 Emissions Per Capita In the India',
                   xaxis_title='Year',
                   yaxis_title='GDP/Capita')

fig.update_layout(
        xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="""
            Year
            <br><br><sup>Correlation Coefficient is 0.972 which proves</sup>
            <br><sup>strong positive correlation for GDP and Co2 Emissions in India</sup>
            """
            )
        )
    )

fig.show()

Putting a price on carbon can encourage low-carbon growth and lower greenhouse gas emissions. Putting a Price Tag on Carbon Reduces Carbon Emission and Supports Economic Growth.

https://www.aa.com.tr/en/energy/analyst/putting-a-price-tag-on-carbon-emissions/13964

**Generated Electricity (TW) 1985- 2020**

In [45]:
energy.head()

,Year,Energy Type,Energy
0,1985,Fossil_Energy,6152.132496
1,1985,Nuclear_Energy,1089.073749
2,1985,Renewable_Electricity,1517.552392
3,1986,Fossil_Energy,6302.889876
4,1986,Nuclear_Energy,1142.099574


In [46]:
fig = px.area(energy, x="Year", y="Energy", color="Energy Type", line_group="Energy Type")

fig.update_layout(title='Generated Electricity (TW) 1985- 2020',
                   xaxis_title='Year',
                   yaxis_title='Generated Electricity (TW)')

fig.show()

**Fossil fuel** has been the world’s primary source of **electricity**.

The majority or 50% of the installed capacity of power plants in Indonesia still comes from fossil energy, especially coal. The need for electrical energy from year to year continues to increase in line with the increasing rate of economic growth, population, and the development of the industrial sector. The use of fossil energy will produce waste in the form of CO2 gas which causes infrared radiation from the earth to return to the earth's surface so that it can cause global warming. In addition, the use of fossil energy as the main source of power generation can also result in depletion of natural resource reserves, such as oil, coal, and gas.

**Annual Global Co2 Emissions from fossil fuel 1900-2020**

In [47]:
co2ann

,Year,Co2Emissions
150,1900,8322746133
151,1901,8503958746
152,1902,8661383395
153,1903,9329177584
154,1904,9463532757
...,...,...
266,2016,124705642926
267,2017,126304712027
268,2018,128452535469
269,2019,128422888228


In [48]:
fig = px.area(co2ann, x="Year", y="Co2Emissions")

fig.update_layout(title='Annual Global Co2 Emissions from fossil fuel 1900-2020',
                   xaxis_title='Year',
                   yaxis_title='CO2 Emissions')

fig.show()